In [1]:
import pandas as pd
import getpass
from functions import connect_mysql, mysql_list
import datetime
import urllib

# Input User id
try:
    # password input hashed
    id = getpass.getpass("id: ") 
except Exception as ex:
    print('Error Occured : ', ex)
#Input password
try:
    # password input hashed
    pwd = getpass.getpass("password: ") 
except Exception as ex:
    print('Error Occured : ', ex)

#host for RDS connection
host = "mdsi-dsp-spr-2020.cehiwpryiego.ap-southeast-2.rds.amazonaws.com"
mydb = connect_mysql(host, id, pwd)

if mydb: 
    print("Connection Successful")
    

id: ········
password: ········
Connection Successful


In [2]:
#today in date format
today = datetime.date.today()
#today in datetime format
today = datetime.datetime(
        year = today.year, 
        month = today.month,
        day = today.day ,
        hour = 11)
period1 = int(today.replace(year = today.year- 5).timestamp()) # 5 years of data
period2 = int(today.timestamp())

print("Period1 : %s" % period1)
print("Period2 : %s" % period2)

params = {"period1": period1,
          "period2": period2,
          "interveral": "1d", # daily price
          "events": "history"
}

Period1 : 1444003200
Period2 : 1601856000


In [3]:
db = "asx"
mydb = connect_mysql(host, id, pwd, db)
df = pd.read_sql_query("SELECT Code FROM asx200", con=mydb)

k = df.Code
v = (df.Code+ ".AX").tolist()
# Create dictionary based on ASX code & yahoo code
asx_index = dict(zip(k, v))
# Add 
asx_index["AXJO"] = "^AXJO" 



In [4]:
# create blank dataframe for data     
asx = pd.DataFrame(columns=["Code", 'Date','Open','High','Low','Close','Adj Close','Volume'])
for key, value in asx_index.items():
    # url for file download request
    url = "https://query1.finance.yahoo.com/v7/finance/download/%s?%s" %(value, urllib.parse.urlencode(params))
    print("Downloading %s: %s" %(list(asx_index.keys()).index(key)+1, key), end="\r")
    with urllib.request.urlopen(url) as f:
        temp = pd.read_csv(f)
        temp['Code'] = key
        asx = asx.append(temp)

# save file in the parent folder
asx.to_csv("../asx_history.csv", index=False)